In [2]:
"""
@author: pho
"""
## Automatic debugger on exception raised
# %pdb on
%pdb off
%load_ext autoreload
%autoreload 2
import sys
import importlib
from pathlib import Path
from copy import deepcopy
from numba import jit
import numpy as np
import pandas as pd

%config Completer.use_jedi = False

# # required to enable non-blocking interaction:
%gui qt5

import os
import numpy as np
import pandas as pd
from pathlib import Path
import dill as pickle

from neuropy.utils.load_exported import LoadXml, find_session_xml # for compute_with_params_loaded_from_xml
# from cnn_ripple.PhoRippleDetectionTesting import ExtendedRippleDetection, main_compute_with_params_loaded_from_xml
from cnn_ripple.PhoRippleDetectionTesting import ExtendedRippleDetection, main_compute_with_params_loaded_from_xml
# from cnn_ripple.PhoRippleDetectionTesting import ExtendedRippleDetection, main_compute_with_params_loaded_from_xml

Automatic pdb calling has been turned OFF


In [8]:
def _perform_load_cnn_computed_ripples(active_local_session_path):
    """
    loaded_ripple_detector = _perform_load_cnn_computed_ripples(curr_active_pipeline.sess.basepath)
    loaded_ripple_detector
    """
    ## Load existing result instead
    in_ripple_detector_filepath = active_local_session_path.joinpath('ripple_detector.pkl') # Path(r'W:\Data\KDIBA\gor01\one\2006-6-07_11-26-53\ripple_detector.pkl')
    loaded_ripple_detector = ExtendedRippleDetection.load(in_ripple_detector_filepath)
    return loaded_ripple_detector

## Continuous prediction probabilities:
def _build_cnn_computed_ripple_prediction_probabilities(loaded_ripple_detector, shank_id: int):  
    """
    ripple_predictions_df, (prediction_timesteps, prediction_values) = _build_cnn_computed_ripple_prediction_probabilities(loaded_ripple_detector, shank_id=2)
    ripple_predictions_df
    """
    # out_all_ripple_results = DynamicContainer.init_from_dict(pd.read_pickle(r'W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\out_all_ripple_results.pkl'))
    out_all_ripple_results = DynamicContainer.init_from_dict(loaded_ripple_detector.out_all_ripple_results.copy())
    # list(out_all_ripple_results.keys()) # ['computation_params', 'results']
    assert shank_id in out_all_ripple_results.results, f"{shank_id} is not in results list: {list(out_all_ripple_results.results.keys())}"
    a_result = out_all_ripple_results.results[shank_id]
    a_result['predictions'].shape # (329925, 1, 1)
    # list(out_all_ripple_results.results.keys()) # [0, 1, 2, 3]
    dt = out_all_ripple_results.computation_params['stride']
    prediction_timesteps = np.arange(np.size(a_result['predictions'])) * dt # + curr_active_pipeline.sess.t_start
    prediction_values = np.squeeze(a_result['predictions'])
    ripple_predictions_df = pd.DataFrame({'t': prediction_timesteps, 'v': prediction_values})
    return ripple_predictions_df, (prediction_timesteps, prediction_values)


def _build_post_load_ripple_df(out_all_ripple_results):
    """ adds the 'shank_idx'
    out_all_ripple_results = loaded_ripple_detector.out_all_ripple_results.copy()
    ripple_df = _build_post_load_ripple_df(out_all_ripple_results)
    """
    out_all_ripple_results_good_results = {k:v for k, v in loaded_ripple_detector.out_all_ripple_results['results'].items() if np.size(v['pred_times'])>0} # Exclude empty items from the output dictionary
    # out_all_ripple_results_good_results
    flattened_pred_ripple_shank_idxs = np.hstack([np.full_like(np.squeeze(a_result['pred_times'][:,0]), a_result['shank'], dtype=np.int16) for a_result in out_all_ripple_results_good_results.values()])
    print(np.shape(flattened_pred_ripple_shank_idxs)) # (6016,)
    flattened_pred_ripple_start_stop_times = np.vstack([a_result['pred_times'] for a_result in out_all_ripple_results_good_results.values() if np.size(a_result['pred_times'])>0])
    print(f'flattened_pred_ripple_start_stop_times: {np.shape(flattened_pred_ripple_start_stop_times)}') # (6498, 2)
    ripple_df = pd.DataFrame({'start':flattened_pred_ripple_start_stop_times[:,0], 'stop': flattened_pred_ripple_start_stop_times[:,1], 'shank_idx': flattened_pred_ripple_shank_idxs})
    ripple_df = ripple_df.sort_values(by=['start']) # sort the values by the start time
    return ripple_df



In [6]:
local_session_parent_path = Path(r'W:\Data\KDIBA\gor01\one')
# local_session_parent_path = Path(r'/Volumes/MoverNew/data/KDIBA/gor01/one')
local_session_names_list = ['2006-6-07_11-26-53', '2006-6-08_14-26-15', '2006-6-09_1-22-43', '2006-6-09_3-23-37', '2006-6-12_15-55-31', '2006-6-13_14-42-6']
local_session_paths_list = [local_session_parent_path.joinpath(a_name).resolve() for a_name in local_session_names_list]

local_session_path: Path = local_session_paths_list[1]
local_session_path

WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15')

## Load previously computed result

In [6]:
loaded_ripple_detector = _perform_load_cnn_computed_ripples(curr_active_pipeline.sess.basepath)
# loaded_ripple_detector

ripple_predictions_df, (prediction_timesteps, prediction_values) = _build_cnn_computed_ripple_prediction_probabilities(loaded_ripple_detector, shank_id=2)
# ripple_predictions_df

# Build the improved ripple_df:
out_all_ripple_results =  DynamicContainer.init_from_dict(loaded_ripple_detector.out_all_ripple_results.copy())
new_ripple_df = _build_post_load_ripple_df(out_all_ripple_results)

## Build and run completely new detector:

In [3]:
session_xml_filepath, session_stem, local_session_path = find_session_xml(local_session_path)
out_xml_dict, d = LoadXml(session_xml_filepath)
active_shank_channels_lists = out_xml_dict['AnatGrps']
# active_shank_channels_lists = [active_shank_channels_lists[i] for i in [0,1,2]]
print(f"active_shank_channels_lists: {active_shank_channels_lists}")

active_shank_channels_lists: [[0, 1, 2, 3, 4, 5, 6, 7], [8, 9, 10, 11, 12, 13, 14, 15], [16, 17, 18, 19, 20, 21, 22, 23], [24, 25, 26, 27, 28, 29, 30, 31], [32, 33, 34, 35, 36, 37, 38, 39], [40, 41, 42, 43, 44, 45, 46, 47], [48, 49, 50, 51, 52, 53, 54, 55], [56, 57, 58, 59, 60, 61, 62, 63], [64, 65, 66, 67, 68, 69, 70, 71], [72, 73, 74, 75, 76, 77, 78, 79], [80, 81, 82, 83, 84, 85, 86, 87], [88, 89, 90, 91, 92, 93, 94, 95]]


In [4]:
test_detector = ExtendedRippleDetection(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False)
ripple_df, out_all_ripple_results = test_detector.compute(**({'active_session_folder': local_session_path,
     'numchannel': out_xml_dict['nChannels'], 'srLfp': out_xml_dict['lfpSampleRate'], 'active_shank_channels_lists': active_shank_channels_lists,
     'overlapping': True, 'window_size': 0.0128, 'window_stride': 0.0064}))

# out_all_ripple_results
ripple_df.to_pickle(local_session_path.joinpath('ripple_df.pkl'))
print(f'done. Exiting.')

test_detector.save()

Loading CNN model... WARNING:tensorflow:SavedModel saved prior to TF 2.5 detected when loading Keras model. Please ensure that you are saving the model with model.save() or tf.keras.models.save_model(), *NOT* tf.saved_model.save(). To confirm, there should be a file named "keras_metadata.pb" in the SavedModel directory.


2022-11-03 12:12:57.272304: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Done!
active_session_folder: /Volumes/MoverNew/data/KDIBA/gor01/one/2006-6-07_11-26-53
active_session_eeg_data_filepath: /Volumes/MoverNew/data/KDIBA/gor01/one/2006-6-07_11-26-53/2006-6-07_11-26-53.eeg
Shape of loaded data:  (2476505, 96)
Downsampling data from 1252 Hz to 1250 Hz... Done!
working on shank 0 with channels: [0, 1, 2, 3, 4, 5, 6, 7]...
Normalizing data... Done!
Shape of loaded data after downsampling and z-score:  (2472549, 8)
Generating windows... Done!
9659/9659 [==============================] - 8s 801us/step
Done!
Getting detected ripples indexes and times... Done!
working on shank 1 with channels: [8, 9, 10, 11, 12, 13, 14, 15]...
Normalizing data... Done!
Shape of loaded data after downsampling and z-score:  (2472549, 8)
Generating windows... Done!
9659/9659 [==============================] - 8s 799us/step
Done!
Getting detected ripples indexes and times... Done!
working on shank 2 with channels: [16, 17, 18, 19, 20, 21, 22, 23]...
Normalizing data... Done!
Shape of

# Result Visualization

In [ ]:
from cnn_ripple.visualizations.plot_ripples import plot_ripples

#@title Time (in seconds) { run: "auto", vertical-output: true, display-mode: "form" }
second =  0#@param {type:"integer"}

pred_indexes = loaded_ripple_detector.results[0]['pred_indexes'] # note only uses shank 0
fig, ax, plot_data = plot_ripples(loaded_ripple_detector.loaded_eeg_data, pred_indexes, second, downsampled_fs=1250)
# plot_data: {'lines':lines, 'fills':fills}

# Overflow

In [ ]:
def keys(a_dict, debug_print=True):
    out_list = list(a_dict.keys())
    if debug_print:
        print(out_list)
    return out_list

In [ ]:
test_detector, ripple_df, out_all_ripple_results, out_all_ripple_results = main_compute_with_params_loaded_from_xml(active_local_session_path)

In [ ]:
out_all_ripple_results['preprocessed_data']['data']

In [ ]:
out_all_ripple_results['computation_params']

In [ ]:
print(list(out_all_ripple_results['computation_params'].keys()))

In [ ]:
keys(out_all_ripple_results['computation_params'])
keys(out_all_ripple_results['preprocessed_data'])
keys(out_all_ripple_results['results'])

In [ ]:
from neuropy.utils.dynamic_container import DynamicContainer, override_dict, overriding_dict_with, get_dict_subset

# ['overlapping', 'window_size', 'stride', 'threshold', 'learning_rate', 'beta_1', 'beta_2', 'epsilon', 'amsgrad']
get_dict_subset(out_all_ripple_results['computation_params'], ['learning_rate', 'beta_1', 'beta_2', 'epsilon', 'amsgrad'])

In [ ]:
test_detector.save()

In [ ]:
loaded_ripple_detector.loaded_eeg_data

In [ ]:
keys(loaded_ripple_detector.results[0]['pred_indexes'])

## Batch resave previous pickles:

In [ ]:
for local_session_path in local_session_paths_list:
    in_ripple_detector_filepath = local_session_path.joinpath('ripple_detector.pkl')
    print(f'in_ripple_detector_filepath: {in_ripple_detector_filepath}')
    loaded_ripple_detector = ExtendedRippleDetection.load(in_ripple_detector_filepath)
    loaded_ripple_detector.save()